In [21]:
library(dplyr)
source('/u/home/m/mikechen/project-xyang123/mergeomics/wrapper_scripts/network2cytoscape.R')
library(stringr)
library(tidyr)
library(tibble)
library(tidyverse)

In [22]:
# kds from modules 2, 0, 5
kds <- read.delim("/u/scratch/m/mikechen/LRRC15/kda/wkda_mod025/all_celllines/kda/wKDA.results.txt")
kds <- kds[which(kds$FDR<0.05),] %>% arrange(FDR)

network <- read.delim("/u/scratch/m/mikechen/LRRC15/kda/networks/all_celllines.txt")
# read in all modules
# modules <- read.delim('/u/scratch/m/mikechen/LRRC15/kda/modules/all_mods.txt')
scing_mods = rbind(read.delim("/u/scratch/m/mikechen/LRRC15/gene_memberships/scing_modules.50.txt"), 
    read.csv("/u/scratch/m/mikechen/LRRC15/gene_memberships/scing_ME_modules.50.subsetted.csv") %>% filter(module %in% c('SCING_M0_hdWGCNA_M1','SCING_M0_deg_0')))
scing_mods$module = gsub('all_','SCING_',scing_mods$module)
scing_mods <- scing_mods %>% group_by(gene_name) %>% summarise(module=paste(module,collapse=';'))%>%ungroup()
# combine module names

hdWGCNA_mods <- read.delim("/u/scratch/m/mikechen/LRRC15/hdwgcna/results/modules/all_celllines_modules.txt")
degs <- read.csv("/u/scratch/m/mikechen/LRRC15/scRNAseq_analysis/samplewise_degs.csv")
degs <- left_join(degs, degs %>% filter(p_val_adj <= 0.05) %>% group_by(gene) %>% summarise(avg_logFC=mean(flipped_avg_log2FC), num_cell_line=length(unique(cell_line))) %>% ungroup() %>% filter(num_cell_line==6) %>% mutate(direction0=ifelse(sign(avg_logFC)==1,'deg0_UP','deg0_DOWN')) %>% select(gene,direction0), by='gene')
degs <- left_join(degs, degs %>% filter(p_val_adj <= 0.05,abs(flipped_avg_log2FC)>=0.5) %>% group_by(gene) %>% summarise(avg_logFC=mean(flipped_avg_log2FC), num_cell_line=length(unique(cell_line))) %>% ungroup() %>% filter(num_cell_line==6) %>% mutate(direction0.5=ifelse(sign(avg_logFC)==1,'deg0.5_UP','deg0.5_DOWN')) %>% select(gene,direction0.5), by='gene')
degs <- degs %>% filter(p_val_adj <= 0.05) %>% 
    group_by(gene) %>% 
    summarise(avg_logFC=mean(flipped_avg_log2FC), num_cell_line=length(unique(cell_line)), direction0=unique(direction0),direction0.5=unique(direction0.5)) %>% ungroup() %>% filter(num_cell_line==6)
# read drug-gene annotations
gene_annot <- read.delim('/u/scratch/m/mikechen/LRRC15/drug_signatures/repositioning_analysis/results/all_module_drug_signatures_082924.txt')

In [23]:
kd_subnetwork <- network[which(network[,1] %in% kds$NODE | network[,2] %in% kds$NODE),]


In [24]:
genes <- unique(c(kd_subnetwork[,1],kd_subnetwork[,2])) # all the genes in the network
# create nodes file
nodes_file <- as.data.frame(genes)
names(nodes_file)[1] <- "NODE"
nodes_file$SHAPE <- "ELLIPSE"


# add in info about whether node is a KD and if subnetwork is shown
adding1 <- add_kd_info(nodes_file,kds,kds)
# identical(nodes_file$NODE,adding1$NODE)
nodes_file <- merge(nodes_file,adding1, by='NODE')

In [25]:
head(nodes_file)

,NODE,SHAPE,KD,KD_subnetwork_shown
,<chr>,<chr>,<lgl>,<chr>
1,ABCD3,ELLIPSE,FALSE,Not a KD
2,ABCF1,ELLIPSE,FALSE,Not a KD
3,ABI1,ELLIPSE,FALSE,Not a KD
4,ABRACL,ELLIPSE,TRUE,TRUE
5,ACAP2,ELLIPSE,FALSE,Not a KD
6,ACAT2,ELLIPSE,TRUE,TRUE


# Drug Annotations

In [26]:
drug_subnetwork <- network[which(network[,1] %in% gene_annot$NODE | network[,2] %in% gene_annot$NODE),]

In [27]:
# add drug annotation
nodes_file <- merge(nodes_file, gene_annot, by="NODE", all=T)

In [28]:
head(nodes_file)

,NODE,SHAPE,KD,KD_subnetwork_shown,Num_Drugs.scing,Drugs.scing,Module.scing,Num_Drugs.hdwgcna,Drugs.hdwgcna,Module.hdwgcna,Num_Drugs.deg0,Drugs.deg0,Module.deg0,Num_Drugs.deg0.5,Drugs.deg0.5,Module.deg0.5,TOTAL_DRUGS,INTERSECT_DRUGS,TOP_DRUGS,num_modules
,<chr>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>
1,AATF,NA,NA,NA,NA,NA,NA,2,"Promethazine,Danazol",all_M18,NA,NA,NA,NA,NA,NA,2,0,"Danazol,Promethazine",1
2,ABCA1,NA,NA,NA,NA,NA,NA,25,"SB-525334,Hydrocortisone,Tyrphostin-AG-1478,Danazol,Tamoxifen,Rottlerin,Temsirolimus,Perphenazine,Pimozide,Maprotiline,Cilnidipine,Budesonide,Bosutinib,GW-5074,Triamcinolone,Loxoprofen,Desipramine,ML-7,TTNPB,Dilazep,Pifithrin-mu,Bexarotene,Chloroquine,Acyclovir,Azithromycin",all_M17,NA,NA,NA,NA,NA,NA,25,0,"Acyclovir,Azithromycin,Bexarotene,Bosutinib,Budesonide,Chloroquine,Cilnidipine,Danazol,Desipramine,Dilazep",1
3,ABCA5,NA,NA,NA,NA,NA,NA,1,Tamoxifen,all_M2,NA,NA,NA,NA,NA,NA,1,0,Tamoxifen,1
4,ABCD3,ELLIPSE,FALSE,Not a KD,11,"Chlorpromazine,Danazol,Temsirolimus,Rottlerin,GW-5074,Budesonide,BW-723C86,Tamoxifen,Promethazine,Azithromycin,Hydrocortisone",SCING_M2,3,"Danazol,BW-723C86,Chlorpromazine",all_M2,NA,NA,NA,NA,NA,NA,11,0,"BW-723C86,Chlorpromazine,Danazol,Azithromycin,Budesonide,GW-5074,Hydrocortisone,Promethazine,Rottlerin,Tamoxifen",2
5,ABCE1,NA,NA,NA,NA,NA,NA,1,Danazol,all_M2,NA,NA,NA,NA,NA,NA,1,0,Danazol,1
6,ABCF1,ELLIPSE,FALSE,Not a KD,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [29]:
nodes_file$SHAPE[is.na(nodes_file$SHAPE)] <- 'ELLIPSE'
nodes_file$KD[is.na(nodes_file$KD)] <- FALSE
nodes_file$KD_subnetwork_shown[is.na(nodes_file$KD_subnetwork_shown)] <- 'Not a KD'
# nodes_file$NUM_DRUGS[is.na(nodes_file$NUM_DRUGS)] <- 0

In [30]:
nodes_file <- nodes_file %>% mutate(Module.scing=merge(nodes_file%>%select(NODE), scing_mods, by.x="NODE", by.y="gene_name", sort=T, all.x = T)%>%pull(module)) %>%
    mutate(Module.hdwgcna=merge(nodes_file%>%select(NODE), hdWGCNA_mods, by.x="NODE", by.y='gene_name', sort=T, all.x=T)%>%pull(module)) %>%
    mutate(Module.deg0=merge(nodes_file%>%select(NODE), degs, by.x="NODE", by.y='gene', sort=T, all.x=T)%>%pull(direction0)) %>%
    mutate(Module.deg0.5=merge(nodes_file%>%select(NODE), degs, by.x="NODE", by.y='gene', sort=T, all.x=T)%>%pull(direction0.5))


In [31]:
head(nodes_file)

,NODE,SHAPE,KD,KD_subnetwork_shown,Num_Drugs.scing,Drugs.scing,Module.scing,Num_Drugs.hdwgcna,Drugs.hdwgcna,Module.hdwgcna,Num_Drugs.deg0,Drugs.deg0,Module.deg0,Num_Drugs.deg0.5,Drugs.deg0.5,Module.deg0.5,TOTAL_DRUGS,INTERSECT_DRUGS,TOP_DRUGS,num_modules
,<chr>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>
1,AATF,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M20,2,"Promethazine,Danazol",all_M18,NA,NA,NA,NA,NA,NA,2,0,"Danazol,Promethazine",1
2,ABCA1,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M18,25,"SB-525334,Hydrocortisone,Tyrphostin-AG-1478,Danazol,Tamoxifen,Rottlerin,Temsirolimus,Perphenazine,Pimozide,Maprotiline,Cilnidipine,Budesonide,Bosutinib,GW-5074,Triamcinolone,Loxoprofen,Desipramine,ML-7,TTNPB,Dilazep,Pifithrin-mu,Bexarotene,Chloroquine,Acyclovir,Azithromycin",all_M17,NA,NA,NA,NA,NA,NA,25,0,"Acyclovir,Azithromycin,Bexarotene,Bosutinib,Budesonide,Chloroquine,Cilnidipine,Danazol,Desipramine,Dilazep",1
3,ABCA5,ELLIPSE,FALSE,Not a KD,NA,NA,NA,1,Tamoxifen,all_M2,NA,NA,NA,NA,NA,NA,1,0,Tamoxifen,1
4,ABCD3,ELLIPSE,FALSE,Not a KD,11,"Chlorpromazine,Danazol,Temsirolimus,Rottlerin,GW-5074,Budesonide,BW-723C86,Tamoxifen,Promethazine,Azithromycin,Hydrocortisone",SCING_M2,3,"Danazol,BW-723C86,Chlorpromazine",all_M2,NA,NA,NA,NA,NA,NA,11,0,"BW-723C86,Chlorpromazine,Danazol,Azithromycin,Budesonide,GW-5074,Hydrocortisone,Promethazine,Rottlerin,Tamoxifen",2
5,ABCE1,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M1,1,Danazol,all_M2,NA,NA,NA,NA,NA,NA,1,0,Danazol,1
6,ABCF1,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M1,NA,NA,all_M7,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [32]:
nodes_file %>% filter(num_modules==4)

NODE,SHAPE,KD,KD_subnetwork_shown,Num_Drugs.scing,Drugs.scing,Module.scing,Num_Drugs.hdwgcna,Drugs.hdwgcna,Module.hdwgcna,Num_Drugs.deg0,Drugs.deg0,Module.deg0,Num_Drugs.deg0.5,Drugs.deg0.5,Module.deg0.5,TOTAL_DRUGS,INTERSECT_DRUGS,TOP_DRUGS,num_modules
<chr>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>
ARF4,ELLIPSE,TRUE,TRUE,6,"Danazol,Tyrphostin-AG-1478,Cilnidipine,Chlorpromazine,Chloroquine,Rottlerin",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,4,"Danazol,Tyrphostin-AG-1478,Chlorpromazine,Rottlerin",all_M1,11,"Tyrphostin-AG-1478,Danazol,Chlorpromazine,Cilnidipine,Rottlerin,Chloroquine,Tamoxifen,Pimozide,Hydrocortisone,Mibefradil,GW-5074",deg0_UP,3,"Cilnidipine,Tyrphostin-AG-1478,Danazol",deg0.5_UP,11,2,"Danazol,Tyrphostin-AG-1478,Chlorpromazine,Cilnidipine,Rottlerin,Chloroquine,GW-5074,Hydrocortisone,Mibefradil,Pimozide",4
CAP1,ELLIPSE,TRUE,TRUE,6,"Rottlerin,Promethazine,GW-5074,Budesonide,Bosutinib,Tamoxifen",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,3,"Promethazine,GW-5074,Rottlerin",all_M1,11,"Promethazine,Triamcinolone,Rottlerin,GW-5074,Bosutinib,Tyrphostin-AG-1478,Tamoxifen,Pimozide,Budesonide,Maprotiline,Pifithrin-mu",deg0_UP,3,"Budesonide,GW-5074,Promethazine",deg0.5_UP,11,2,"GW-5074,Promethazine,Budesonide,Rottlerin,Bosutinib,Tamoxifen,Maprotiline,Pifithrin-mu,Pimozide,Triamcinolone",4
EID1,ELLIPSE,TRUE,TRUE,3,"Cilnidipine,RO-90-7501,TTNPB",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,1,Perphenazine,all_M1,19,"RO-90-7501,Cilnidipine,Triamcinolone,Hydrocortisone,SB-525334,Budesonide,Arvanil,Desipramine,Perphenazine,Tamoxifen,Clemastine,Acyclovir,Bosutinib,Chlorpromazine,TTNPB,Azithromycin,GW-5074,Rottlerin,Desmethylclozapine",deg0_UP,5,"Cilnidipine,Hydrocortisone,Budesonide,GW-5074,Desipramine",deg0.5_UP,19,0,"Cilnidipine,Budesonide,Desipramine,GW-5074,Hydrocortisone,Perphenazine,RO-90-7501,TTNPB,Acyclovir,Arvanil",4
FKBP1A,ELLIPSE,TRUE,TRUE,9,"Mibefradil,Tyrphostin-AG-1478,Hydrocortisone,Chloroquine,Tamoxifen,Chlorpromazine,Danazol,Temsirolimus,TTNPB",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,1,Tyrphostin-AG-1478,all_M1,17,"Tyrphostin-AG-1478,Hydrocortisone,Danazol,Rottlerin,Chloroquine,Pifithrin-mu,Mibefradil,Temsirolimus,Tamoxifen,GW-5074,Tenidap,Chlorpromazine,Budesonide,TTNPB,Triamcinolone,Cilnidipine,Pimozide",deg0_UP,5,"Chlorpromazine,Tyrphostin-AG-1478,Hydrocortisone,Danazol,Budesonide",deg0.5_UP,17,1,"Tyrphostin-AG-1478,Chlorpromazine,Danazol,Hydrocortisone,Budesonide,Chloroquine,Mibefradil,TTNPB,Tamoxifen,Temsirolimus",4
H2AFV,ELLIPSE,TRUE,TRUE,18,"Rottlerin,Promethazine,Danazol,Dilazep,Perphenazine,Budesonide,Tamoxifen,SB-525334,RO-90-7501,Triamcinolone,Tyrphostin-AG-1478,GW-5074,Clomipramine,Desipramine,Chlorpromazine,Icariin,Hydrocortisone,Cilnidipine",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,6,"Danazol,Promethazine,Hydrocortisone,Perphenazine,Tamoxifen,Rottlerin",all_M1,24,"Danazol,GW-5074,Dilazep,Triamcinolone,Promethazine,Acyclovir,Tamoxifen,Tyrphostin-AG-1478,SB-525334,Hydrocortisone,Desipramine,Rottlerin,Cilnidipine,Clomipramine,Chlorpromazine,RO-90-7501,Perphenazine,Mibefradil,Pimozide,Budesonide,Icariin,Chloroquine,TTNPB,Tenidap",deg0_UP,12,"Acyclovir,Chlorpromazine,Danazol,Hydrocortisone,Desipramine,Budesonide,Rottlerin,Icariin,Dilazep,GW-5074,Tyrphostin-AG-1478,Promethazine",deg0.5_UP,24,4,"Danazol,Hydrocortisone,Promethazine,Rottlerin,Budesonide,Chlorpromazine,Desipramine,Dilazep,GW-5074,Icariin",4
HSBP1,ELLIPSE,TRUE,TRUE,6,"Promethazine,Bexarotene,Rottlerin,Tamoxifen,Hydrocortisone,GW-5074",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,2,"Promethazine,Tamoxifen",all_M1,14,"Bexarotene,Promethazine,Rottlerin,Tamoxifen,Chlorpromazine,ML-7,Triamcinolone,Hydrocortisone,GW-5074,Cilnidipine,Chloroquine,Budesonide,Pifithrin-mu,Azithromycin",deg0_UP,3,"Promethazine,GW-5074,Rottlerin",deg0.5_UP,14,1,"Promethazine,GW-5074,Rottlerin,Tamoxifen,Bexarotene,Hydrocortisone,Azithromycin,Budesonide,Chloroquine,Chlorpromazine",4
IQGAP1,ELLIP

In [33]:
top_kds <- c(rep(T,50),rep(F,nrow(kds)-50))
names(top_kds) <- kds$NODE
nodes_file$topKD <- top_kds[nodes_file$NODE]

In [34]:
head(drug_subnetwork)

,HEAD,TAIL,WEIGHT
,<chr>,<chr>,<dbl>
21,RARG,C1orf53,1926.300
31,SLC35A3,TGFBRAP1,1789.915
35,CXCL1,IL8,1748.634
37,TUBB4B,TUBA1C,1729.344
46,EIF3J-AS1,RNF113A,1643.786
48,FGF1,OXTR,1591.415


In [35]:
outdir <- 'results'
write.table(kd_subnetwork, paste0(outdir,'/kd_edges_depth1_all_mods_082924.txt'), sep='\t', row.names=F, quote=F)
write.table(drug_subnetwork, paste0(outdir,'/drug_edges_depth1_all_mods_082924.txt'), sep='\t', row.names=F, quote=F)
write.table(nodes_file, paste0(outdir,'/drug_kd_nodes_all_mods_082924.txt'), sep='\t', row.names=F, quote=F)


In [36]:
combined_network <- unique(rbind(kd_subnetwork, drug_subnetwork))

In [37]:
write.table(combined_network, paste0(outdir,'/drug_kd_edges_depth1_all_mods_082924.txt'), sep='\t', row.names=F, quote=F)


In [38]:
nodes_file %>% filter(Module.scing=="SCING_M29")

NODE,SHAPE,KD,KD_subnetwork_shown,Num_Drugs.scing,Drugs.scing,Module.scing,Num_Drugs.hdwgcna,Drugs.hdwgcna,Module.hdwgcna,⋯,Drugs.deg0,Module.deg0,Num_Drugs.deg0.5,Drugs.deg0.5,Module.deg0.5,TOTAL_DRUGS,INTERSECT_DRUGS,TOP_DRUGS,num_modules,topKD
<chr>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<lgl>
ACOT9,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,3,"Tyrphostin-AG-1478,Danazol,Perphenazine",all_M1,⋯,NA,NA,NA,NA,NA,3,0,"Danazol,Perphenazine,Tyrphostin-AG-1478",1,NA
BABAM1,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,2,"Danazol,Tamoxifen",all_M1,⋯,NA,NA,NA,NA,NA,2,0,"Danazol,Tamoxifen",1,NA
CD97,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,2,"Rottlerin,Hydrocortisone",all_M1,⋯,"Hydrocortisone,Danazol,Rottlerin,Tyrphostin-AG-1478,Chloroquine,GW-5074,Pimozide,Maprotiline,Budesonide,Pifithrin-mu,Bosutinib,Clomipramine,Acyclovir,Tamoxifen",deg0_UP,NA,NA,NA,14,0,"Hydrocortisone,Rottlerin,Acyclovir,Bosutinib,Budesonide,Chloroquine,Clomipramine,Danazol,GW-5074,Maprotiline",2,NA
CGGBP1,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,NA,NA,all_M22,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CHCHD7,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,NA,NA,all_M22,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KIAA1429,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,NA,NA,all_M14,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
LPAR1,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,NA,NA,all_M22,⋯,"Rottlerin,Hydrocortisone,Tamoxifen,Maprotiline,Chlorpromazine,Tyrphostin-AG-1478,Cilnidipine,Perphenazine,Bosutinib,SB-525334,Budesonide,Promethazine,BW-723C86,GW-5074,Arvanil,Pimozide,ML-7,TTNPB,Triamcinolone,Tenidap,Temsirolimus,Desipramine,Chloroquine,Pifithrin-mu,Mibefradil,Acyclovir,Dilazep,Flupentixol,Desmethylclozapine",deg0_UP,NA,NA,NA,29,0,"Acyclovir,Arvanil,BW-723C86,Bosutinib,Budesonide,Chloroquine,Chlorpromazine,Cilnidipine,Desipramine,Desmethylclozapine",1,NA
MAGEF1,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,1,Danazol,all_M1,⋯,NA,NA,NA,NA,NA,1,0,Danazol,1,NA
MRPS28,ELLIPSE,FALSE,Not a KD,NA,NA,SCING_M29,NA,NA,all_M22,⋯,"Danazol,Azithromycin,Chloroquine,Rottlerin,Pimozide,Hydrocortisone",deg0_UP,NA,NA,NA,6,0,"Azithromycin,Chloroquine,Danazol,Hydrocortisone,Pimozide,Rottlerin",1,NA


In [39]:
nodes_file %>% filter(num_modules==4)

NODE,SHAPE,KD,KD_subnetwork_shown,Num_Drugs.scing,Drugs.scing,Module.scing,Num_Drugs.hdwgcna,Drugs.hdwgcna,Module.hdwgcna,⋯,Drugs.deg0,Module.deg0,Num_Drugs.deg0.5,Drugs.deg0.5,Module.deg0.5,TOTAL_DRUGS,INTERSECT_DRUGS,TOP_DRUGS,num_modules,topKD
<chr>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<lgl>
ARF4,ELLIPSE,TRUE,TRUE,6,"Danazol,Tyrphostin-AG-1478,Cilnidipine,Chlorpromazine,Chloroquine,Rottlerin",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,4,"Danazol,Tyrphostin-AG-1478,Chlorpromazine,Rottlerin",all_M1,⋯,"Tyrphostin-AG-1478,Danazol,Chlorpromazine,Cilnidipine,Rottlerin,Chloroquine,Tamoxifen,Pimozide,Hydrocortisone,Mibefradil,GW-5074",deg0_UP,3,"Cilnidipine,Tyrphostin-AG-1478,Danazol",deg0.5_UP,11,2,"Danazol,Tyrphostin-AG-1478,Chlorpromazine,Cilnidipine,Rottlerin,Chloroquine,GW-5074,Hydrocortisone,Mibefradil,Pimozide",4,FALSE
CAP1,ELLIPSE,TRUE,TRUE,6,"Rottlerin,Promethazine,GW-5074,Budesonide,Bosutinib,Tamoxifen",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,3,"Promethazine,GW-5074,Rottlerin",all_M1,⋯,"Promethazine,Triamcinolone,Rottlerin,GW-5074,Bosutinib,Tyrphostin-AG-1478,Tamoxifen,Pimozide,Budesonide,Maprotiline,Pifithrin-mu",deg0_UP,3,"Budesonide,GW-5074,Promethazine",deg0.5_UP,11,2,"GW-5074,Promethazine,Budesonide,Rottlerin,Bosutinib,Tamoxifen,Maprotiline,Pifithrin-mu,Pimozide,Triamcinolone",4,FALSE
EID1,ELLIPSE,TRUE,TRUE,3,"Cilnidipine,RO-90-7501,TTNPB",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,1,Perphenazine,all_M1,⋯,"RO-90-7501,Cilnidipine,Triamcinolone,Hydrocortisone,SB-525334,Budesonide,Arvanil,Desipramine,Perphenazine,Tamoxifen,Clemastine,Acyclovir,Bosutinib,Chlorpromazine,TTNPB,Azithromycin,GW-5074,Rottlerin,Desmethylclozapine",deg0_UP,5,"Cilnidipine,Hydrocortisone,Budesonide,GW-5074,Desipramine",deg0.5_UP,19,0,"Cilnidipine,Budesonide,Desipramine,GW-5074,Hydrocortisone,Perphenazine,RO-90-7501,TTNPB,Acyclovir,Arvanil",4,FALSE
FKBP1A,ELLIPSE,TRUE,TRUE,9,"Mibefradil,Tyrphostin-AG-1478,Hydrocortisone,Chloroquine,Tamoxifen,Chlorpromazine,Danazol,Temsirolimus,TTNPB",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,1,Tyrphostin-AG-1478,all_M1,⋯,"Tyrphostin-AG-1478,Hydrocortisone,Danazol,Rottlerin,Chloroquine,Pifithrin-mu,Mibefradil,Temsirolimus,Tamoxifen,GW-5074,Tenidap,Chlorpromazine,Budesonide,TTNPB,Triamcinolone,Cilnidipine,Pimozide",deg0_UP,5,"Chlorpromazine,Tyrphostin-AG-1478,Hydrocortisone,Danazol,Budesonide",deg0.5_UP,17,1,"Tyrphostin-AG-1478,Chlorpromazine,Danazol,Hydrocortisone,Budesonide,Chloroquine,Mibefradil,TTNPB,Tamoxifen,Temsirolimus",4,FALSE
H2AFV,ELLIPSE,TRUE,TRUE,18,"Rottlerin,Promethazine,Danazol,Dilazep,Perphenazine,Budesonide,Tamoxifen,SB-525334,RO-90-7501,Triamcinolone,Tyrphostin-AG-1478,GW-5074,Clomipramine,Desipramine,Chlorpromazine,Icariin,Hydrocortisone,Cilnidipine",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,6,"Danazol,Promethazine,Hydrocortisone,Perphenazine,Tamoxifen,Rottlerin",all_M1,⋯,"Danazol,GW-5074,Dilazep,Triamcinolone,Promethazine,Acyclovir,Tamoxifen,Tyrphostin-AG-1478,SB-525334,Hydrocortisone,Desipramine,Rottlerin,Cilnidipine,Clomipramine,Chlorpromazine,RO-90-7501,Perphenazine,Mibefradil,Pimozide,Budesonide,Icariin,Chloroquine,TTNPB,Tenidap",deg0_UP,12,"Acyclovir,Chlorpromazine,Danazol,Hydrocortisone,Desipramine,Budesonide,Rottlerin,Icariin,Dilazep,GW-5074,Tyrphostin-AG-1478,Promethazine",deg0.5_UP,24,4,"Danazol,Hydrocortisone,Promethazine,Rottlerin,Budesonide,Chlorpromazine,Desipramine,Dilazep,GW-5074,Icariin",4,FALSE
HSBP1,ELLIPSE,TRUE,TRUE,6,"Promethazine,Bexarotene,Rottlerin,Tamoxifen,Hydrocortisone,GW-5074",SCING_M0;SCING_M0_hdWGCNA_M1;SCING_M0_deg_0,2,"Promethazine,Tamoxifen",all_M1,⋯,"Bexarotene,Promethazine,Rottlerin,Tamoxifen,Chlorpromazine,ML-7,Triamcinolone,Hydrocortisone,GW-5074,Cilnidipine,Chloroquine,Budesonide,Pifithrin-mu,Azithromycin",deg0_UP,3,"Promethazine,GW-5074,Rottlerin",deg0.5_UP,14,1,"Promethazine,GW-5074,Rottlerin,Tamoxifen,Bexarotene,Hydrocortisone,Azithromycin,Budesonide,Chloroquine,Chlorpromaz